In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), # 28>28 | 1>3 | 1>1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0),
            nn.Conv2d(16, 16, 3, padding=1), # 28>28 | 3>5 | 1>1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.),
            nn.Conv2d(16, 16, 3, padding=1), # 28>28 | 5>7 | 1>1
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.) # 28>14 | 7>8 | 1>2
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1), # 14>14 | 8>12 | 2>2
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.),
            nn.Conv2d(16, 16, 3), # 14>12 | 12>16 | 2>2
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.),
            nn.Conv2d(16, 16, 3), # 12>10 | 16>20 | 2>2
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.),
            nn.MaxPool2d(2, 2) # 10>5 | 24>26 | 2>4
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3), # 5>3 | 26>30 | 4>4
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.),
        )
        self.fc = nn.Sequential(
            #nn.Linear(3*3*32, 10)
            nn.Conv2d(32, 10, 1),
            nn.AvgPool2d(3)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        #x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [12]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           2,320
              ReLU-6           [-1, 16, 28, 28]               0
       BatchNorm2d-7           [-1, 16, 28, 28]              32
           Dropout-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 16, 28, 28]           2,320
             ReLU-10           [-1, 16, 28, 28]               0
      BatchNorm2d-11           [-1, 16, 28, 28]              32
        MaxPool2d-12           [-1, 16, 14, 14]               0
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-15., 15.), fill=0),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0525, Accuracy: 9871/10000 (98.71%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0316, Accuracy: 9907/10000 (99.07%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0207, Accuracy: 9945/10000 (99.45%)



  0%|                                                                                          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0192, Accuracy: 9932/10000 (99.32%)



loss=0.008067495189607143 batch_id=468: 100%|████████████████████████████████████████| 469/469 [00:10<00:00, 45.78it/s]



Test set: Average loss: 0.0188, Accuracy: 9951/10000 (99.51%)

